In [1]:
import pandas as pd
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score, GridSearchCV, train_test_split, StratifiedKFold
from sklearn.metrics import confusion_matrix, recall_score, roc_auc_score, accuracy_score, f1_score
from sklearn.model_selection import cross_val_score
from imblearn.pipeline import Pipeline, make_pipeline


In [2]:
X, y = make_classification(n_samples=12500, n_classes=2,n_features=10, weights=[0.99, 0.01], flip_y=0, random_state=2020)

## Splitting the dataset into the Training set and Test set


In [3]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 2020,stratify=y)

In [4]:
# Results from split
train_0, train_1 = len(y_train[y_train==0]), len(y_train[y_train==1])
test_0, test_1 = len(y_test[y_test==0]), len(y_test[y_test==1])
print('>Train: 0=%d, 1=%d, Test: 0=%d, 1=%d' % (train_0, train_1, test_0, test_1))


>Train: 0=9900, 1=100, Test: 0=2475, 1=25


## Creating CrossValidation Object

In [5]:
from sklearn.model_selection import StratifiedKFold 
kf = StratifiedKFold(n_splits=5, random_state=2020, shuffle=True)


## SMOTE Oversampling for the Minority Class

In [6]:
from collections import Counter
from imblearn.over_sampling import SMOTE
# summarize class distribution
print(Counter(y_train))
# define oversample strategy
oversample = SMOTE(sampling_strategy=0.5)
# fit and apply the transform
X_sample, y_sample = oversample.fit_resample(X_train, y_train)
# summarize class distribution
print(Counter(y_sample))

Counter({0: 9900, 1: 100})
Counter({0: 9900, 1: 4950})


## Training the Logistic Regression model on the Training set

In [7]:
from sklearn.linear_model import LogisticRegression
lm = LogisticRegression(random_state = 2020)
lm.fit(X_sample, y_sample)

LogisticRegression(random_state=2020)

## Confusion Matrix

In [8]:
y_pred = lm.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[2380   95]
 [   6   19]]


In [9]:
#Warning: Do not sample the test set on cross validation
pipeline = make_pipeline(oversample, 
                              lm)
# Applying k-Fold Cross Validation
score = cross_val_score(pipeline, X_train, y_train, scoring='recall', cv=kf)
print('>Mean recall: %.1f, std: %.1f' % (score.mean()*100, score.std()*100))

>Mean recall: 83.0, std: 8.7


## What happens if we sample BEFORE Cross-validation?
Biased or over optimistic results

In [10]:
# This is a mistake
score = cross_val_score(lm, X_sample, y_sample, cv=kf, scoring='recall')
print('>Mean recall - WARNING: %.1f, std - WARNING: %.1f' % (score.mean()*100, score.std()*100))


>Mean recall - WARNING: 88.7, std - WARNING: 1.1


## Training the SVM model on the Training set


In [11]:
from sklearn.svm import SVC
svm = SVC(kernel = 'linear', random_state = 0)
svm.fit(X_sample, y_sample)

SVC(kernel='linear', random_state=0)

## Confusion Matrix

In [12]:
y_pred = svm.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[2372  103]
 [   5   20]]


In [21]:
#Warning: Do not sample the test set on cross validation
pipeline = make_pipeline(oversample, 
                              svm)
# Applying k-Fold Cross Validation
score = cross_val_score(pipeline, X_train, y_train, scoring='recall', cv=kf)
print('>Mean recall: %.1f, std: %.1f' % (score.mean()*100, score.std()*100))

>Mean recall: 83.0, std: 8.7


## Training the Random Forest model on the Training set


In [14]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 2020)
rf.fit(X_sample, y_sample)

RandomForestClassifier(criterion='entropy', n_estimators=10, random_state=2020)

## Confusion Matrix

In [15]:
y_pred = rf.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[2459   16]
 [   8   17]]


In [16]:
#Warning: Do not sample the test set on cross validation
pipeline = make_pipeline(oversample, 
                              rf)
# Applying k-Fold Cross Validation
score = cross_val_score(pipeline, X_train, y_train, scoring='recall', cv=kf)
print('>Mean recall: %.1f, std: %.1f' % (score.mean()*100, score.std()*100))

>Mean recall: 79.0, std: 11.1


## Training the ANN model on the Training set

In [17]:
import keras
from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf

# Initialising the ANN
ann = Sequential()

# Adding the input layer and the first hidden layer
ann.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 10))

# Adding the second hidden layer
ann.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))

# Adding the output layer
ann.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

# Compiling the ANN
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# Fitting the ANN to the Training set
ann.fit(X_sample, y_sample, batch_size = 10, epochs = 10)



Using TensorFlow backend.


Epoch 1/10
14850/14850 [==============================] - 8s 512us/step - loss: 0.2525 - accuracy: 0.9040
Epoch 2/10
14850/14850 [==============================] - 6s 414us/step - loss: 0.1485 - accuracy: 0.9451
Epoch 3/10
14850/14850 [==============================] - 8s 513us/step - loss: 0.1290 - accuracy: 0.9526
Epoch 4/10
14850/14850 [==============================] - 5s 363us/step - loss: 0.1193 - accuracy: 0.9555
Epoch 5/10
14850/14850 [==============================] - 5s 337us/step - loss: 0.1140 - accuracy: 0.9573
Epoch 6/10
14850/14850 [==============================] - 6s 435us/step - loss: 0.1098 - accuracy: 0.9587
Epoch 7/10
14850/14850 [==============================] - 6s 388us/step - loss: 0.1061 - accuracy: 0.9609
Epoch 8/10
14850/14850 [==============================] - 7s 475us/step - loss: 0.1036 - accuracy: 0.9611
Epoch 9/10
14850/14850 [==============================] - 5s 366us/step - loss: 0.1010 - accuracy: 0.9630
Epoch 10/10
14850/14850 [=====================

## Confusion Matrix

In [19]:
# Part 3 - Making the predictions and evaluating the model
# Predicting the Test set results
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[2374  101]
 [   7   18]]


In [20]:
#Applying Kfold CrossValidation
# Evaluating the ANN
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from keras.models import Sequential
from keras.layers import Dense

def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 10))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier
classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, epochs = 10)

#Warning: Do not sample the test set on cross validation
pipeline = make_pipeline(oversample, 
                              classifier)

score = cross_val_score(pipeline, X = X_train, y = y_train, cv = kf, n_jobs = -1,scoring='recall')
print('>Mean recall: %.1f, std: %.1f' % (score.mean()*100, score.std()*100))

>Mean recall: 79.0, std: 3.7
